### Linear regression - model specification
This notebook demostrates:

0. Correctly specified model (baseline)
1. Under-specified model
    - Uncorrelated predictors
    - Correlated predictors
2. Over-specified model
    - Perfect multi-colinearity
    - Varying degrees of Multi-colinearity
    - Including non related variables

In [1]:
import numpy as np
import statsmodels.api as sm

### 1. Correctly specified model (baseline)

- We randomly generate 100 uncorrelated X1 and X2 following a normal distirbution.

- We speficy a true regression line: $y$ = 3 + 2*$X_1$ + 4*$X_2$ + e

- We correctly include X1 and X2 in our regression model: y ~ X1 + X2 (with intercept)

- We fit 1000 such regression models and check for biasedness and preciseness of the estimates.

In [2]:
estimates = []
for i in range(1000):
    X1 = np.random.randn(100) + 2
    X2 = np.random.randn(100) + 2
    
    e = np.random.randn(100)
    y = 3 + 2*X1 + 4*X2 + e
    X = sm.add_constant(np.column_stack([X1,X2]))
    model = sm.OLS(y, X).fit()
    estimates.append(model.params)

In [3]:
#The averages of estimates across the 1000 replications are:
print("Average of estimates:", np.mean(np.array(estimates),axis=0))
print("SE of estimates:", np.std(np.array(estimates),axis=0))

Average of estimates: [3.0037811  2.00347394 3.99617821]
SE of estimates: [0.30086517 0.09971629 0.10145132]


### 1. Under-specified model

#### Uncorrelated predictors (#1)
- We randomly generate 100 uncorrelated X1 and X2 following a normal distirbution.

- We speficy a true regression line: $y$ = 3 + 2*$X_1$ + 4*$X_2$ + e

- We only include X1 in our regression model: y ~ X1 (with intercept)

- We fit 1000 such regression models and check for biasedness.

In [4]:
estimates = []
for i in range(1000):
    means = [2 , 2]
    cov = [[1,0], [0,1]]
    
    X = np.random.multivariate_normal(means,cov,100)

    e = np.random.randn(100)
    
    y = 3 + 2*X[:,0] + 4*X[:,1] + e
    
    X = sm.add_constant(X[:,0])
    model = sm.OLS(y, X).fit()
    
    estimates.append(model.params)

In [5]:
#The averages of estimates across the 1000 replications are:
print("Average of estimates:", np.mean(np.array(estimates),axis=0))

print("SE of estimates:", np.std(np.array(estimates),axis=0))

Average of estimates: [11.01614906  1.99128492]
SE of estimates: [0.95786376 0.42621356]


We can see the intercept estimate is **biased** (if will be unbiased only if your omitted variable has a mean of zero), and the coefficient for b1 is **unbiased**. The SE of the estimates are much higher than that in a correctly specified model.

#### Correlated predictors (#2)
- We randomly generate 100 correlated X1 and X2 following a bi-variate normal distirbution (so X1 and X2 are having a correlation coefficient of 0.4).

- We speficy a true regression line: $y$ = 3 + 2*$X_1$ + 4*$X_2$ + e

- We only include X1 in our regression model: y ~ X1 (with intercept)

- We fit 1000 such regression models and check for biasedness.

In [6]:
estimates = []
for i in range(1000):
    means = [2, 2]
    cov = [[1,0.4], [0.4,1]]
    X = np.random.multivariate_normal(means,cov,100)

    e = np.random.randn(100)
    y = 3 + 2*X[:,0] + 4*X[:,1] + e
    X = sm.add_constant(X[:,0])
    model = sm.OLS(y, X).fit()
    estimates.append(model.params)

In [7]:
#The averages of estimates across the 1000 replications are:
np.mean(np.array(estimates),axis=0)

array([7.77702431, 3.60628815])

In [8]:
#The averages of estimates across the 1000 replications are:
np.std(np.array(estimates),axis=0)

array([0.85773046, 0.38963273])

We can see the intercept estimate is **biased** (it will be unbiased only if both of your variable have a mean of zero), and the coefficient for b1 is also **biased** from its true value of 2.

Conclusion: when predictors are correlated, omitting one will make other estimates biased.

### 2. Over-specified model

#### Perfect multicolinearity (#1)
- We randomly generate 100 data points for X1 following a normal distribution.
- We set X2 = 1 - X1

- We speficy a true regression line: $y$ = 3 + 2*$X_1$ + 4*$X_2$ + e

- We only fit the regression model as: y ~ X1 + X2 (with intercept)


In [9]:
X1 = np.random.randn(100)
X2 = 1 - X1
e = np.random.randn(100)

y = 3 + 2*X1 + 4*X2 + e

X = sm.add_constant(np.column_stack([X1,X2]))

model = sm.OLS(y,X).fit()

In [10]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     466.2
Date:                Tue, 13 Feb 2024   Prob (F-statistic):           4.98e-39
Time:                        09:50:43   Log-Likelihood:                -141.90
No. Observations:                 100   AIC:                             287.8
Df Residuals:                      98   BIC:                             293.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0724      0.075     54.100      0.000       3.923       4.222
x1             1.0145      0.072     14.005      0.000       0.871       1.158
x2             3.0579      0.045     67.296      0.000       2.968       3.148
==============================================================================
Omnibus:                        1.402   Durbin-Watson:                   2.156
Prob(Omnibus):                  0.496   Jarque-Bera (JB):                1.313
Skew:                           0.152   Prob(JB):                        0.519
Kurtosis:                       2.527   Cond. No.                     1.32e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.91e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

####  Multicolinearity (#2)
- We randomly generate 100 correlated X1 and X2 following a bi-variate normal distirbution and let X1 and X2 have different degree of correlation (0, 0.3, 0.6, 0.9, 0.95, and 0.99).

- We speficy a true regression line: $y$ = 2*$X_1$ + 4*$X_2$ + e

- We include X1 and X2 in our regression model: y ~ X1 + X2 (no intercept, for simplicity)

- For each pre-set correlation value, we repeat 1000 times to generate the sampling distirbution of regression estimates.
- Plot the sampling distirbutions

In [18]:
#Write a function to return the sampling distribution of regression coefficients 
# based on different degree of correlation between X1 and X2
def simulation_multi_colinearity(cor):
    params = []
    for i in range(1000):
        means = [2,2]
        cov = [[1,cor], [cor,1]]
        X = np.random.multivariate_normal(means,cov,100)
        e = np.random.randn(100)*2
        y = 2*X[:,0] + 4*X[:,1] + e
        model = sm.OLS(y,X).fit()
        params.append(model.params)
    return params

In [19]:
sampling_dist_0 = simulation_multi_colinearity(0)
sampling_dist_0_3 = simulation_multi_colinearity(0.3)
sampling_dist_0_6 = simulation_multi_colinearity(0.6)
sampling_dist_0_9 = simulation_multi_colinearity(0.9)
sampling_dist_0_95 = simulation_multi_colinearity(0.95)
sampling_dist_0_99 = simulation_multi_colinearity(0.99)

In [20]:
print("SE of estimates when X1 and X2 have a cor of 0:", np.std(sampling_dist_0,axis=0))
print("SE of estimates when X1 and X2 have a cor of 0.3:", np.std(sampling_dist_0_3,axis=0))
print("SE of estimates when X1 and X2 have a cor of 0.6:", np.std(sampling_dist_0_6,axis=0))
print("SE of estimates when X1 and X2 have a cor of 0.9:", np.std(sampling_dist_0_9,axis=0))
print("SE of estimates when X1 and X2 have a cor of 0.95:", np.std(sampling_dist_0_95,axis=0))
print("SE of estimates when X1 and X2 have a cor of 0.99:", np.std(sampling_dist_0_99,axis=0))

SE of estimates when X1 and X2 have a cor of 0: [0.14924138 0.15362695]
SE of estimates when X1 and X2 have a cor of 0.3: [0.18184739 0.18149639]
SE of estimates when X1 and X2 have a cor of 0.6: [0.23542159 0.23302496]
SE of estimates when X1 and X2 have a cor of 0.9: [0.46005811 0.46522827]
SE of estimates when X1 and X2 have a cor of 0.95: [0.62053723 0.61689469]
SE of estimates when X1 and X2 have a cor of 0.99: [1.5058649  1.50165901]


In [22]:
print("Mean of estimates when X1 and X2 have a cor of 0:", np.mean(sampling_dist_0,axis=0))
print("Mean of estimates when X1 and X2 have a cor of 0.3:", np.mean(sampling_dist_0_3,axis=0))
print("Mean of estimates when X1 and X2 have a cor of 0.6:", np.mean(sampling_dist_0_6,axis=0))
print("Mean of estimates when X1 and X2 have a cor of 0.9:", np.mean(sampling_dist_0_9,axis=0))
print("Mean of estimates when X1 and X2 have a cor of 0.95:", np.mean(sampling_dist_0_95,axis=0))
print("Mean of estimates when X1 and X2 have a cor of 0.99:", np.mean(sampling_dist_0_99,axis=0))

Mean of estimates when X1 and X2 have a cor of 0: [1.99528853 4.00220908]
Mean of estimates when X1 and X2 have a cor of 0.3: [1.99374527 4.00339456]
Mean of estimates when X1 and X2 have a cor of 0.6: [2.01461324 3.98911849]
Mean of estimates when X1 and X2 have a cor of 0.9: [2.02487827 3.97569048]
Mean of estimates when X1 and X2 have a cor of 0.95: [2.04199266 3.95822687]
Mean of estimates when X1 and X2 have a cor of 0.99: [1.96516269 4.03487359]


We can observe that, with varying degrees of correlation between the two predictors in the model, the regression coefficients are still **unbiased**, but the standard error of the estimates are **inflated** based on the degree of the correlation.

#### Adding non-related predictors (#2)
- We randomly generate 100 data points for X1, X2, and X3 following a normal distribution.

- We speficy a true regression line: $y$ = 3 + 2*$X_1$ + 4*$X_2$ + e

- We  include X1, X2, and X3in our regression model: y ~ X1 (with intercept). Here X3 should not be in the model, but included.

- We fit 1000 such regression models and check for biasedness.

In [15]:
estimates = []
for i in range(1000):
    X1 = np.random.randn(100)
    X2 = np.random.randn(100)
    X3 = np.random.randn(100)
    e = np.random.randn(100)

    y = 3 + 2*X1 + 4*X2 + e

    X = sm.add_constant(np.column_stack([X1,X2,X3]))
    model = sm.OLS(y, X).fit()
    estimates.append(model.params)

In [16]:
#The averages of estimates across the 1000 replications are:
print("Average of estimates:", np.mean(np.array(estimates),axis=0))
print("SE of estimates:", np.std(np.array(estimates),axis=0))

Average of estimates: [ 3.00132214e+00  1.99576168e+00  3.99886082e+00 -2.36659646e-03]
SE of estimates: [0.10146959 0.09980508 0.10321398 0.1018764 ]


We find the estimates of X1 and X2 are **unbiased**, and the standard errors are also quite small. The estimate for X3 is nearly **zero**, and if we check one model out of the 1000, we find the estimate is not statistically significant. In this sense, including non-related random data into the model does not do  much harm.

In [17]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                     806.9
Date:                Tue, 13 Feb 2024   Prob (F-statistic):           6.27e-68
Time:                        09:50:44   Log-Likelihood:                -138.60
No. Observations:                 100   AIC:                             285.2
Df Residuals:                      96   BIC:                             295.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0338      0.099     30.704      0.000       2.838       3.230
x1             1.9591      0.094     20.781      0.000       1.772       2.146
x2             3.9472      0.090     43.918      0.000       3.769       4.126
x3             0.1564      0.099      1.580      0.117      -0.040       0.353
==============================================================================
Omnibus:                        2.692   Durbin-Watson:                   2.382
Prob(Omnibus):                  0.260   Jarque-Bera (JB):                2.525
Skew:                          -0.387   Prob(JB):                        0.283
Kurtosis:                       2.920   Cond. No.                         1.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""